# Create Internet radio buttons in Domoticz

In [ ]:
import base64
import json
from urllib.request import urlopen
from pprint import pprint

## Obtain all internet radio links

Using links from the 6-7 regions in EU:
https://www.hendrikjansen.nl/henk/streaming.html

In [ ]:
# To collect all radio stations and save in "radio_stations.json": 
from collect_iradio_links import collect_stations, urls
pprint(urls)

The variable urls gives a specification how to collect a subset internet radio links from the long lists in the websites. Regular expressions are used for selections and some namings are cleaned up after scanning the html code.

In [ ]:
radio_stations = collect_stations(urls)
pprint(radio_stations)

### Your own selection
This was a small example, now create your own selection by modifying the original selection. Just add regular expressions to the list per EU region.

Please be aware that depending the website you want to use you may need to modify the functions _collect_stations_ in [collect_iradio_links.py](file:collect_iradio_links.py).

In [ ]:
urls = {
    'https://www.hendrikjansen.nl/henk/streaming.html':
    ('NL', [
        'NPO [3R].*',  # select all NPO radio stations
        ]),
    'https://www.hendrikjansen.nl/henk/streaming1.html':
    ('BE LU', [
        'Radio [12]$',  # select only 'Radio 1' and 'Radio 2'
        '.*Klara.*',    # Select all stations matching 'Klara' 
        ]),
    # 'https://www.hendrikjansen.nl/henk/streaming3.html':  # DU LI AU
    # 'https://www.hendrikjansen.nl/henk/streaming2.html':  # UK IR
    # 'https://www.hendrikjansen.nl/henk/streaming4.html':  # FR ES BU
    # 'https://www.hendrikjansen.nl/henk/streaming5.html':  # AF NA
    # 'https://www.hendrikjansen.nl/henk/streaming6.html':
    # 'https://www.hendrikjansen.nl/henk/streaming7.html':  # NO
}
radio_stations = collect_stations(urls)
# Save the radio urls (not needed)
with open('radio_stations.json', 'w') as f:
    print(json.dumps(radio_stations, sort_keys=True, indent=4), file=f)

## Integrate in domoticz
For selecting a radiostations selector buttons are used. **Before running this part create the selector buttons in domoticz by hand**.

Here we modify the selector buttons using a json url request. This url is partially base64 encoded (for the button options) so two variables are used to construct the url.

In [ ]:
from create_domoticz_selector import selectorDef, selectorOpts, cr_url

baseurl = 'http://localhost/'  # My local domoticz url

In [ ]:
pprint(selectorDef)

**Important:** the _idx_ variable must be correct: it is the switch id in domoticz which was created before. (*TODO:* Maybe there exists a method to just create a new switch??)

In [ ]:
pprint(selectorOpts)

Now we can test the reprogramming of the selector switch: it should return with 

"status": "OK"

In [ ]:
url = cr_url(baseurl, selectorDef, selectorOpts)
with urlopen(url) as f:
    print(f.read().decode('utf-8'))

### My own selectors
The variable _radio_stations_ contains per region (ex. _NL_ and _BE LU_) a list of tuples with the station name and streaming url. We define _selectors_ as the list of selector button, with their _idx_ and radio station selections. Here: selector switches with _idx_: 256-257 are defined.

In [ ]:
selectors = {
    'Radio NPO': (256, [
        ('NL', 'NPO Radio 1'),
        ('NL', 'NPO Radio 2'),
        ('NL', 'NPO Radio 2 Soul & Jazz'),
        ('NL', 'NPO 3FM'),
        ('NL', 'NPO Radio 4'),
        ('NL', 'NPO Radio 5')
    ]),
    'Radio Vlaanderen': (257, [
        ('BE LU', 'Klara'),
        ('BE LU', 'Klara Continuo'),
        ('BE LU', 'Radio 1'),
        ('BE LU', 'Radio 2'),
    ]),
}
print(sorted(selectors.keys()))
print([v[0] for k, v in selectors.items()])

In [ ]:
s = selectorDef
o = selectorOpts
for m in selectors:
    s['idx'] = str(selectors[m][0])
    s['name'] = m
    o['LevelNames'] = ['Off'] + [r[1] for r in selectors[m][1]]
    o['LevelActions'] = [''] * len(o['LevelNames'])
    url = cr_url(baseurl, s, o)
    print(url)
    with urlopen(url) as f:
        print(f.read().decode('utf-8'))

## dzVents lua script

A script is required which acts on events of the selector switches. Obviously selecting an internet radio station requires starting of **mplayer** and the amplifier with speakers should be connected.

The [script radio.lua](file:radio.lua) uses a lua table which is generated here.

In [ ]:
print('local stations = {')
for m in selectors:
    for c, l in selectors[m][1]:
        #  entry = "{'%s', '%s'}," % (l, radio_stations[c][l])
        entry = "    ['%s'] = '%s'," % (l, radio_stations[c][l])  # Create lua table
        print(entry)
print('}')

The script needs to be modified to your own taste. In this example an Onkyo player is used which also happens to be used as a bluetooth player.

Things to modify:
- Add the stations table
- Add the domoticz devices you have defined in the _on_ section and list them in the variable _selectors_ in section _execute_
- Modify the _execute_ section to your likings

**Important note**: when making a selection in one selector you want all other selectors being switched off. As a consequence all **mplayer** processes are killed. Since these switch-off events are scheduled and therefore executed later, the start of the internet radio stream needs to be delayed (with a second in this case) in _os.execute_.